In [1]:
!pip install openai
!pip install langchain
!pip install pysrt

  Using cached idna-3.6-py3-none-any.whl (61 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl (162 kB)


You should consider upgrading via the 'C:\Users\Homeuser\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)


You should consider upgrading via the 'C:\Users\Homeuser\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Using legacy 'setup.py install' for pysrt, since package 'wheel' is not installed.
    Running setup.py install for pysrt: started
    Running setup.py install for pysrt: finished with status 'done'


You should consider upgrading via the 'C:\Users\Homeuser\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
input1="東門市場是濟州島三大市場中最大也是最好逛的一個，連接七星商店街、中央地下街，構成濟州島最繁華的區域，濟州島的海鮮、漁貨特別新鮮，市場裡可以嚐到許多海鮮醃漬品和傳統小吃，基本的辣炒年糕、紫菜包飯、魚板等都有，另外任何濟州島名產，像柑橘製品、綠茶、巧克力和石頭爺爺紀念品都能買到。韓國服飾、流行物件、小文具則集中在七星商店街跟中央地下街，價格比首爾便宜很多，喜愛逛街的遊客絕對要來！"
input2="既然換機油很重要，很多人也關心機車換機油里程數、更換週期的問題，那麼究竟機車黑油多久換一次？機車機油多久換才正確呢？由於不同車款的引擎運轉速度不同、不同車主的使用習慣也不同，建議你先了解自己的機車使用情況！首先，你可以根據「使用情況」判斷，若是常常怠速、走走停停的短程代步會使引擎內的溫度一直很高，造成機油的效能下降，就需要更頻繁換機油，比如台灣人常有的騎車習慣，包括兩段式左轉、在菜市場騎車購物等。其次是評估「騎車環境」，比如台灣氣候溼熱，機油更易變質；或若常前往海邊或風沙大的地方也應該要更頻繁更換機油。還有可以視「車況、車種」而定，像是時常有人詢問「新車多久換機油、新車機油多久換一次」，基本上新機車換機油頻率可參考機車原廠說明書，一般大多是300至500公里換第1次機油，1,000公里時換第2次機油，接著就從自己的使用情況來判斷，約1,000至2,000公里進行更換較佳。而重機機油多久換呢？重機的更換頻率比一般機車來得低，除了檢查機油尺外，若感受到卡檔也可進行更換，約3,000至5,000公里換1次即可。最後你可以看「機油種類」，通常礦物油的更換頻率大於全合成機油。基於很多人也常問，機車齒輪油多久換一次？為了方便記憶，只要記得「每換2次機油，就換1次齒輪油」就可以了喔！"

把逐字稿丟進去，先去掉多餘的字再變成列點式。
輸入API KEY及input，執行overall_chain.run(input)。

In [4]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
llm = OpenAI(temperature=1, openai_api_key="keeeeey")
## 建立模板
template = '''去掉這段話的冗言贅字
% TRANSCRIPT
{transcript}

你的回复:
'''
prompt_template = PromptTemplate(input_variables=["transcript"], template=template)
# 建立location的chain
chain1 = LLMChain(llm=llm, prompt=prompt_template)
## 接下来获取chain的返回结果输入到下一个chain中

writingstemplate = '''把這段話變列點式、
% writings
{writings}

您的回复:
'''
prompt_template = PromptTemplate(input_variables=["writings"], template=writingstemplate)
chain1 = LLMChain(llm=llm, prompt=prompt_template)
# 建立meal的chain
chain2 = LLMChain(llm=llm, prompt=prompt_template)
# 建立简单链
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
outcome = overall_chain.run(input1)



> Entering new SimpleSequentialChain chain...
• 東門市場是濟州島三大市場裡最大且最值得逛的一個，聯外七星商店街和中央地下街，構成濟州島最繁華的區域。
• 濟州島新鮮的海鮮及漁貨，市場裡可以品嚐到許多海鮮醃漬品和傳統小吃，像辣炒年糕、紫菜包飯、魚板等等。
• 濟州島的名產，如柑
• 東門市場: 最大且最值得逛的三大市場之一
• 聯外七星商店街和中央地下街: 構成濟州島最繁華的區域
• 海鮮及漁貨: 可在市場中品嚐到許多海鮮醃漬品和傳統小吃
• 柑: 濟州島的名產

> Finished chain.


把整理好的筆記跟user的筆記做對比，補足缺少部分

In [5]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # 两个input_key的并集
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}
prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="请给我這段話的列點式整理 {product} ",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="请给我這段話的大意 {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)
## 连接chain
concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run(input1)
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


1. 東門市場：最大、最好逛的濟州島三大市場 
2. 連接七星商店街、中央地下街，構成濟州島最繁華的區域
3. 海鮮、漁貨新鮮、可嚐許多海鮮醃漬品和傳統小吃 
4. 辣炒年糕、紫菜包飯、魚板以及濟州島名產：柑橘製品、綠茶、巧克力等
5. 七星商店街、

東門市場是濟州島最大也是最繁華的區域，在市場裡可以買到新鮮的海鮮、漁貨以及傳統濟州島小吃，並且可以買到很多濟州島名產，另外在七星商店街以及中央地下街也能买到韓國服飾、流行物件等價格比首爾低很多，很適合喜愛逛街的遊客。
